In [1]:
import xarray as xr
import numpy as np
import glob
import os.path

from dask.distributed import Client
from dask.diagnostics import ProgressBar

from aggfly import dataset, regions, grid_weights
from aggfly.aggregate import TemporalAggregator, SpatialAggregator, get_time_dim

ProgressBar().register()
# client = Client()

In [ ]:
# Set file output name/path
output_path = "/home3/dth2133/data/aggregated/counties/"
output_name = "usa_counties_corn_monthly"
output_varn = "prec"
csv = True

In [65]:
# Open shapefile containing region features.
georegions = regions.from_name('counties')

# Open example climate dataset to calculate grid weights.
clim = dataset.from_path(
    f"/home3/dth2133/data/usa/usa-tp_tempPrecLand2019.zarr", 
    'tp', 
    'zarr', 
    preprocess=dataset.timefix_era5l)

# Clip climate data to the US (raw data are global)
clim.clip_data_to_georegions_extent(georegions)

# Rechunk dataset to optimize multithreading
clim.rechunk((5, 578, -1, -1, -1, -1))

# Calculate area and crop layer weights.
weights = grid_weights.from_objects(clim, georegions, crop='corn')

# This object covers aggregating hourly and daily data to the yearly 
# level
# daily = [TemporalAggregator(
#     'dd',
#     agg_from='hour',
#     agg_to='day', 
#     ddargs=[x,999,0]) for x in range(-3,36)]

annual = TemporalAggregator(
    'sum', 
    agg_from='hour',
    agg_to='month')

# This object aggregates cells within a region to the average across 
# cells, weighted by `weights`, which in this case are the area of the
# cell and the share of the cell with corn crops.
spatial = SpatialAggregator('avg')


TypeError: __init__() missing 1 required positional argument: 'agg_from'

In [4]:
# Calculate the grid weights
w = weights.weights()

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 27.7s
[########################################] | 100% Completed | 26.2s
[########################################] | 100% Completed | 42.4s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s


In [26]:
# import importlib
# import aggfly
# from aggfly.aggregate.temporal import TemporalAggregator
# importlib.reload(aggfly.aggregate.temporal)
# from aggfly.aggregate.temporal import TemporalAggregator

In [28]:
import dask
def timefix_era5l_tp(array):
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):   
        array['year'] = array.time.dt.year
        array['month'] = array.time.dt.month
        array['day'] = array.time.dt.day
        array['hour'] = array.time.dt.hour
        array = array.set_index(time=("year", "month", "day", "hour")).unstack('time')
        # array = array - 273.15
    return array

In [29]:
def aggregate_era5l_tp(path):
    # path = "/home3/dth2133/data/usa/usa-t2m_tempPrecLand1969.zarr"
    # Open climate dataset.
    clim = dataset.from_path(
        path, 
        'tp', 
        'zarr', 
        preprocess=timefix_era5l_tp) # Kelvin to Celsius
    # print(clim)
    # Clip climate data to the US (raw data are global)
    clim.clip_data_to_georegions_extent(georegions)
    # Rechunk dataset to optimize multithreading
    clim.rechunk((5, 578, -1, -1, -1))

    # Update climate dataset in `clim` to one collapsed over
    # hour and day based upon `temporal` definition above.
    # clim = daily.map_execute(clim)
    annual.map_execute(clim, update=True)
    clim.rechunk(-1)

    # Again update, but with (weighted) spatial collapse over regions.
    spatial.map_execute(clim, w)

    # Send back the aggregated climate data.
    return clim.da


In [30]:
# Check years from the input path - this just creates a vector of years for
# which my raw climate data are available, e.g. [1970, 1971, ...]
import numpy as np
import glob
from os.path import basename
files = np.sort([x for x in glob.glob('/home3/dth2133/data/usa/*tp_*')])
# Loop over years and aggregate.
output = list()
for f in files:
    print(f)
    output.append(aggregate_era5l_tp(f))

/home3/dth2133/data/usa/usa-tp_tempPrecLand1951.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1952.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1953.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1954.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1955.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1956.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1957.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1958.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1960.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1961.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1962.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1963.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1964.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1965.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1966.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1967.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1968.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1969.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand197

In [ ]:
da = xr.concat(output, dim='year').compute()

[##########                              ] | 26% Completed | 10min 26.5s

In [44]:
outds = da.to_dataset(name=output_varn)

In [61]:
ds = xr.open_zarr(os.path.join(output_path, output_name+'.zarr'))
ds = ds.drop(output_varn)
ds = xr.combine_by_coords([ds,outds])

In [63]:
ds = ds.compute()
ds

<xarray.Dataset>
Dimensions:  (region: 3108, year: 68, month: 12)
Coordinates:
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * region   (region) <U5 '01001' '01003' '01005' ... '56041' '56043' '56045'
  * year     (year) int64 1951 1952 1953 1954 1955 ... 2015 2016 2017 2018 2019
Data variables: (12/49)
    dd08     (region, year, month) float64 97.64 128.0 216.3 ... 5.062 0.1649
    dd10     (region, year, month) float64 64.49 94.92 167.7 ... 2.201 0.01085
    dd29     (region, year, month) float64 0.0 0.0 0.0 0.0009577 ... 0.0 0.0 0.0
    dd30     (region, year, month) float64 0.0 0.0 0.0 0.0 ... 1.215 0.0 0.0 0.0
    dd36     (region, year, month) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    prec     (region, year, month) float64 1.122 1.45 1.851 ... 0.4905 0.1758
    ...       ...
    time7    (region, year, month) float64 19.73 18.38 26.26 ... 2.677 0.2933
    time8    (region, year, month) float64 18.33 17.48 25.25 ... 1.945 0.1252
    time9    (region, year, month) float64 16.62 16.52 24.27 ... 1.42 0.06682
    timeM1   (region, year, month) float64 30.02 24.82 31.0 ... 15.51 9.025
    timeM2   (region, year, month) float64 30.38 25.27 31.0 ... 18.21 10.96
    timeM3   (region, year, month) float64 30.86 25.95 31.0 ... 20.55 13.45

In [60]:
# https://github.com/pydata/xarray/issues/3476
for v in list(ds.coords.keys()):
    if ds.coords[v].dtype == object:
        ds.coords[v] = ds.coords[v].astype("unicode")

for v in list(ds.variables.keys()):
    if ds[v].dtype == object:
        ds[v] = ds[v].astype("unicode")
        
ds.to_zarr(os.path.join(output_path, output_name+'.zarr'), mode='w')

In [64]:
if csv:
    # ds = xr.open_zarr(os.path.join(output_path, output_name+'.zarr'))
    ds.to_dataframe().to_csv(os.path.join(output_path, output_name+'.csv'))

In [41]:
xr.open_zarr(os.path.join(output_path, output_name+'.zarr'))

<xarray.Dataset>
Dimensions:  (region: 3108, year: 68, month: 12)
Coordinates:
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * region   (region) object '21007' '21017' '21031' ... '48171' '55079' '26139'
  * year     (year) int64 1951 1952 1953 1954 1955 ... 2015 2016 2017 2018 2019
Data variables: (12/49)
    dd08     (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    dd10     (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    dd29     (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    dd30     (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    dd36     (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    prec     (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    ...       ...
    time7    (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    time8    (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    time9    (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    timeM1   (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    timeM2   (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    timeM3   (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>

In [16]:
# Rename for stata analysis
ds = xr.open_zarr(os.path.join(output_path, output_name+'.zarr'))
ds.to_dataframe().to_csv(os.path.join(output_path, output_name+'.csv'))

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

In [16]:
output_name

'usa_counties_corn_monthly'

In [15]:
ds

<xarray.Dataset>
Dimensions:  (region: 3108, year: 68, month: 12)
Coordinates:
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * region   (region) object '21007' '21017' '21031' ... '48171' '55079' '26139'
  * year     (year) int64 1951 1952 1953 1954 1955 ... 2015 2016 2017 2018 2019
Data variables: (12/44)
    dd08     (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    dd10     (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    dd29     (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    dd30     (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    dd36     (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    time0    (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    ...       ...
    time7    (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    time8    (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    time9    (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    timeM1   (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    timeM2   (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>
    timeM3   (region, year, month) float64 dask.array<chunksize=(777, 17, 6), meta=np.ndarray>